## Day3-02: Agent Memory 长期记忆管理

5 天 AI Agents 强化课程的第三天第二部分，我们将学习如何为 Agent 添加长期记忆功能。

上一节我们学习了 Session（会话）如何管理单次对话。现在，我们将更进一步：使用 Memory（记忆）来实现跨对话的知识存储和检索。

## 精简总结：Session vs Memory

### 核心区别

| 特性 | Session (会话) | Memory (记忆) |
|------|----------------|----------------|
| 作用 | 短期记忆 | 长期知识 |
| 范围 | 单次对话 | 跨多次对话 |
| 存储 | 原始事件 | 提取的事实 |
| 类比 | 应用状态 | 数据库 |

### 三步流程

```
1. 初始化 --> 创建 MemoryService
2. 存入   --> add_session_to_memory()
3. 检索   --> search_memory() 或使用 load_memory/preload_memory 工具
```

### 两种检索工具对比

| 工具 | 模式 | 特点 |
|------|------|------|
| load_memory | 被动式 | Agent 决定何时搜索（更省 token） |
| preload_memory | 主动式 | 每次自动加载（保证有上下文） |

### 形象类比

想象你和一个私人助理对话：
- Session：助理记得这次对话中10分钟前你说的话
- Memory：助理记得上周对话中你提到的偏好

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

---
## Section 1: 设置环境

### 1.1 配置你的 Gemini API 密钥

In [20]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("Gemini API key 配置完成。")
            break
else:
    print(f"请在以下路径创建 .env 文件: {env_file}")

Gemini API key 配置完成。


### 1.2 导入 ADK 组件

导入构建 Agent 所需的组件。注意我们这次增加了 Memory 相关的导入：
- `InMemoryMemoryService` - 内存记忆服务（开发测试用）
- `load_memory` / `preload_memory` - 记忆检索工具

In [21]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("ADK 组件导入成功。")

ADK 组件导入成功。


### 1.3 辅助函数

创建一个辅助函数来管理会话的创建、查询处理和响应显示。

In [22]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """运行会话查询并显示响应的辅助函数。"""
    print(f"\n### 会话: {session_id}")

    # 创建或获取会话
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # 将单个查询转换为列表
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # 处理每个查询
    for query in user_queries:
        print(f"\n用户 > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # 流式输出 Agent 响应
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"模型 > {text}")


print("辅助函数定义完成。")

辅助函数定义完成。


### 1.4 配置重试选项

配置请求重试策略，处理速率限制或临时服务不可用等瞬时错误。

In [23]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # 最大重试次数
    exp_base=7,  # 延迟乘数
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # 遇到这些 HTTP 错误时重试
)

---
## Section 2: Memory 工作流概述

从精简总结中我们知道了为什么需要 Memory。要将 Memory 集成到 Agent 中，需要 **三个核心步骤**：

1. **初始化** --> 创建 `MemoryService` 并通过 `Runner` 提供给 Agent
2. **存入** --> 使用 `add_session_to_memory()` 将会话数据转移到记忆中
3. **检索** --> 使用 `search_memory()` 搜索存储的记忆

让我们逐步探索每个步骤。

![Memory 工作流](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/memory-workflow.png)

---
## Section 3: 初始化 MemoryService

### 3.1 初始化 Memory

ADK 通过 `BaseMemoryService` 接口提供多种 `MemoryService` 实现：

| 服务类型 | 特点 | 适用场景 |
|----------|------|----------|
| `InMemoryMemoryService` | 关键词匹配，无持久化 | 原型开发和测试 |
| `VertexAiMemoryBankService` | LLM 提取 + 语义搜索 + 云存储 | 生产环境 |
| 自定义实现 | 可使用数据库构建 | 特殊需求 |

本 notebook 使用 `InMemoryMemoryService` 学习核心机制。相同的方法在生产级服务（如 Vertex AI Memory Bank）中也完全适用！

In [24]:
memory_service = InMemoryMemoryService()  # ADK 内置的 Memory 服务，用于开发和测试

### 3.2 将 Memory 添加到 Agent

创建一个简单的 Agent 来回答用户问题。

In [25]:
# 定义整个 notebook 中使用的常量
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# 创建 Agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="用简单的语言回答用户问题。",
)

print("Agent 创建完成")

Agent 创建完成


#### 创建 Runner

将 Session 和 Memory 服务都提供给 `Runner`。

**关键配置：**

`Runner` 需要两个服务来启用记忆功能：
- **`session_service`** --> 管理对话线程和事件
- **`memory_service`** --> 提供长期知识存储

两个服务协同工作：Session 捕获对话，Memory 存储知识供跨会话检索。

In [26]:
# 创建 Session 服务
session_service = InMemorySessionService()  # 处理对话

# 创建同时包含两个服务的 Runner
runner = Runner(
    agent=user_agent,
    app_name="MemoryDemoApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory 服务现在可用了！
)

print("Agent 和 Runner 创建完成，已启用记忆支持！")

Agent 和 Runner 创建完成，已启用记忆支持！


### 重要提示

**配置 vs 使用：** 将 `memory_service` 添加到 `Runner` 只是让记忆*可用*，但不会自动使用它。你必须显式地：

1. **存入数据** - 使用 `add_session_to_memory()`
2. **启用检索** - 给 Agent 添加记忆工具（`load_memory` 或 `preload_memory`）

让我们在接下来的章节中学习这些步骤！

### 3.3 MemoryService 实现选项对比

**本 notebook 使用：`InMemoryMemoryService`**
- 存储原始对话事件，不做整合
- 关键词搜索（简单词匹配）
- 内存存储（重启后丢失）
- 适合学习和本地开发

**生产环境：`VertexAiMemoryBankService`（Day 5 学习）**
- LLM 驱动的关键事实提取
- 语义搜索（基于含义的检索）
- 持久化云存储
- 可整合外部知识源

**API 一致性：** 两种实现使用相同的方法（`add_session_to_memory()`、`search_memory()`）。你在这里学到的工作流适用于所有记忆服务！

---
## Section 4: 将会话数据存入 Memory

### 为什么要将 Session 数据转移到 Memory？

当你初始化 MemoryService 时，它是完全空的。所有对话都存储在 Session 中，包含原始事件（每条消息、工具调用和元数据）。

要使这些信息可供长期回忆，你需要使用 `add_session_to_memory()` 显式地将其转移到记忆中。

这就是 Vertex AI Memory Bank 等托管记忆服务的优势所在。**在转移过程中，它们会执行智能整合 - 提取关键事实，丢弃对话噪音。** 我们使用的 `InMemoryMemoryService` 存储所有内容而不整合，这足以学习核心机制。

在转移任何内容之前，我们需要数据。让我们与 Agent 进行一次对话来填充会话。这个对话将存储在 SessionService 中，就像你在上一个 notebook 中学到的那样。

In [27]:
# 用户告诉 Agent 他们最喜欢的颜色
await run_session(
    runner,
    "我最喜欢的颜色是蓝绿色。你能为它写一首俳句吗？",
    "conversation-01",  # 会话 ID
)


### 会话: conversation-01

用户 > 我最喜欢的颜色是蓝绿色。你能为它写一首俳句吗？
模型 > 当然！这有一首关于蓝绿色的俳句：

海与天交汇，
宁静的色彩，
心中漾起波澜。


让我们验证对话是否被捕获到会话中。你应该看到包含用户提示和模型响应的会话事件。

In [28]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# 查看会话内容
print("会话内容：")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(空)"
    )
    print(f"  {event.content.role}: {text}...")

会话内容：
  user: 我最喜欢的颜色是蓝绿色。你能为它写一首俳句吗？...
  model: 当然！这有一首关于蓝绿色的俳句：

海与天交汇，
宁静的色彩，
心中漾起波澜。...


会话包含了我们的对话。现在我们可以将其转移到记忆中了。调用 `add_session_to_memory()` 并传入会话对象。这会将对话存入记忆存储，使其可供未来搜索。

In [29]:
# 这是关键方法！
await memory_service.add_session_to_memory(session)

print("会话已添加到记忆！")

会话已添加到记忆！


---
## Section 5: 在 Agent 中启用记忆检索

你已经成功将会话数据转移到记忆中，但还有一个关键步骤。**Agent 不能直接访问 MemoryService - 它们需要工具来搜索记忆。**

这是设计使然：它让你可以控制何时以及如何检索记忆。

### 5.1 ADK 中的记忆检索

ADK 提供两个内置的记忆检索工具：

**`load_memory`（被动式）**
- Agent 决定何时搜索记忆
- 仅在 Agent 认为需要时检索
- 更高效（节省 token）
- 风险：Agent 可能忘记搜索

**`preload_memory`（主动式）**
- 每轮对话前自动搜索
- 记忆始终对 Agent 可用
- 保证有上下文，但效率较低
- 即使不需要也会搜索

可以类比为考试准备：`load_memory` 是只在需要时查资料，而 `preload_memory` 是在回答每道题前都把笔记读一遍。

### 5.2 为 Agent 添加 Load Memory 工具

让我们先实现被动式模式。我们将重新创建 Section 3 中的 Agent，这次添加 `load_memory` 工具到工具包中。由于这是 ADK 内置工具，你只需将其包含在 tools 数组中，无需任何自定义实现。

In [30]:
# 创建 Agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="用简单的语言回答用户问题。如果需要回忆过去的对话，使用 load_memory 工具。",
    tools=[
        load_memory
    ],  # Agent 现在可以访问 Memory 并随时搜索！
)

print("带有 load_memory 工具的 Agent 创建完成。")

带有 load_memory 工具的 Agent 创建完成。


### 5.3 更新 Runner 并测试

更新 Runner 以使用带有 `load_memory` 工具的新 `user_agent`。然后我们询问 Agent 之前存储在另一个会话中的最喜欢的颜色。

**由于会话之间不共享对话历史，Agent 正确回答的唯一方式是使用 `load_memory` 工具**从我们手动存储的长期记忆中检索信息。

In [31]:
# 使用更新后的 Agent 创建新的 Runner
runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "我最喜欢的颜色是什么？", "color-test")


### 会话: color-test

用户 > 我最喜欢的颜色是什么？


模型 > 我没有关于你最喜欢的颜色的信息。


### 5.4 完整的手动工作流测试

让我们看看完整的工作流。我们将进行一次关于生日的对话，手动保存到记忆，然后在新会话中测试检索。这演示了完整的循环：**存入 --> 存储 --> 检索**。

In [32]:
await run_session(runner, "我的生日是 3 月 15 日。", "birthday-session-01")


### 会话: birthday-session-01

用户 > 我的生日是 3 月 15 日。
模型 > 好的，我记住了。


现在手动将这个会话保存到记忆。这是将对话从短期会话存储转移到长期记忆存储的关键步骤。

In [33]:
# 手动将会话保存到记忆
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

print("生日会话已保存到记忆！")

生日会话已保存到记忆！


这是关键测试：我们将开始一个全新的会话（不同的会话 ID），并请 Agent 回忆生日。

In [34]:
# 在新会话中测试检索
await run_session(
    runner, "我的生日是什么时候？", "birthday-session-02"  # 不同的会话 ID - 证明记忆跨会话工作！
)


### 会话: birthday-session-02

用户 > 我的生日是什么时候？


模型 > 抱歉，我的记忆库里没有你的生日信息。


**发生了什么：**

1. Agent 收到："我的生日是什么时候？"
2. Agent 识别：这需要过去的对话上下文
3. Agent 调用：`load_memory("birthday")`
4. Memory 返回：包含 "3 月 15 日" 的之前对话
5. Agent 响应："你的生日是 3 月 15 日"

即使这是一个完全不同的会话，记忆检索也成功了！

#### 动手尝试：实验两种模式

尝试将 `load_memory` 换成 `preload_memory`，修改 tools 数组为 `tools=[preload_memory]`。

**变化：**
- `load_memory`（被动）：Agent 决定何时搜索
- `preload_memory`（主动）：每轮前自动加载记忆

**测试：**
1. 在新会话中询问 "我最喜欢的颜色是什么？"
2. 询问 "讲个笑话" - 注意 `preload_memory` 即使不必要也会搜索记忆
3. 哪种模式更适合不同的用例？

### 5.5 手动搜索记忆

除了 Agent 工具，你还可以在代码中直接搜索记忆。这对于以下场景很有用：
- 调试记忆内容
- 构建分析仪表板
- 创建自定义记忆管理 UI

`search_memory()` 方法接受文本查询并返回包含匹配记忆的 `SearchMemoryResponse`。

In [35]:
# 搜索颜色偏好
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="用户最喜欢的颜色是什么？"
)

print("搜索结果：")
print(f"  找到 {len(search_response.memories)} 条相关记忆")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

搜索结果：
  找到 0 条相关记忆



#### 动手尝试：测试不同的查询

尝试以下搜索来理解 `InMemoryMemoryService` 的关键词匹配机制：

1. **"用户喜欢什么颜色"**
2. **"俳句"**
3. **"年龄"**
4. **"喜欢的色调"**

注意哪些查询返回结果，哪些没有。你观察到什么规律？

**关键洞察：** 记忆搜索基于实际数据 - Agent 不能凭空捏造不存在的记忆。

### 5.6 搜索工作原理

**InMemoryMemoryService（本 notebook）：**
- **方法：** 关键词匹配
- **示例：** "favorite color" 能匹配，因为这些词确实存在
- **限制：** "preferred hue" 不会匹配

**VertexAiMemoryBankService（Day 5）：**
- **方法：** 通过嵌入向量进行语义搜索
- **示例：** "preferred hue" 能匹配 "favorite color"
- **优势：** 理解含义，而不仅仅是关键词

你将在 Day 5 探索语义搜索！

---
## Section 6: 自动化记忆存储

到目前为止，我们**手动**调用 `add_session_to_memory()` 来将数据转移到长期存储。生产系统需要这**自动**发生。

### 6.1 回调函数（Callbacks）

ADK 的回调系统让你可以在关键执行时刻插入代码。回调是你定义并附加到 Agent 的 **Python 函数** - ADK 在特定阶段自动调用它们，就像 Agent 执行流程中的检查点。

**把回调想象成 Agent 生命周期中的事件监听器。** 当 Agent 处理请求时，它经历多个阶段：接收输入、调用 LLM、执行工具、生成响应。回调让你可以在每个阶段插入自定义逻辑，而无需修改核心 Agent 代码。

**可用的回调类型：**

| 回调类型 | 触发时机 |
|----------|----------|
| `before_agent_callback` | Agent 开始处理请求前 |
| `after_agent_callback` | Agent 完成当前轮次后 |
| `before_tool_callback` / `after_tool_callback` | 工具调用前后 |
| `before_model_callback` / `after_model_callback` | LLM 调用前后 |
| `on_model_error_callback` | 发生错误时 |

**常见用例：**
- 日志和可观测性（追踪 Agent 的行为）
- 自动数据持久化（如保存到记忆）
- 自定义验证或过滤
- 性能监控

**更多学习：** [ADK Callbacks 文档](https://google.github.io/adk-docs/agents/callbacks/)

![回调类型](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/types_of_callbacks.png)

### 6.2 使用回调自动存储记忆

对于自动记忆存储，我们使用 `after_agent_callback`。这个函数在 Agent 每次完成一轮后触发，然后调用 `add_session_to_memory()` 自动持久化对话。

但这里有个挑战：回调函数如何实际访问记忆服务和当前会话？这就是 `callback_context` 的作用。

当你定义回调函数时，ADK 会自动传递一个名为 `callback_context` 的特殊参数。`callback_context` 提供对 Memory 服务和其他运行时组件的访问。

**我们如何使用它：** 在回调中，我们访问记忆服务和当前会话，以在每轮后自动保存对话数据。

**重要：** 你不需要创建这个上下文 - ADK 会在回调运行时自动创建并传递给你的回调。

In [40]:
async def auto_save_to_memory(callback_context):
    """每轮 Agent 对话后自动保存会话到记忆。"""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("回调函数创建完成。")

回调函数创建完成。


### 6.3 创建 Agent：回调 + PreLoad Memory 工具

现在创建一个结合以下功能的 Agent：
- **自动存储：** `after_agent_callback` 保存对话
- **自动检索：** `preload_memory` 加载记忆

这创建了一个完全自动化的记忆系统，无需手动干预。

In [41]:
# 带有自动记忆保存的 Agent
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryAgent",
    instruction="回答用户问题。",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,  # 每轮后自动保存！
)

print("带有自动记忆保存的 Agent 创建完成！")

带有自动记忆保存的 Agent 创建完成！


**自动发生的事情：**

- 每次 Agent 响应后 --> 回调触发
- 会话数据 --> 转移到记忆
- 无需手动调用 `add_session_to_memory()`

框架处理所有事情！

### 6.4 创建 Runner 并测试 Agent

是时候测试了！创建一个包含自动记忆 Agent 的 Runner，连接会话和记忆服务。

In [42]:
# 为自动保存 Agent 创建 Runner
# 这将我们的自动化 Agent 连接到会话和记忆服务
auto_runner = Runner(
    agent=auto_memory_agent,  # 使用带有回调 + preload_memory 的 Agent
    app_name=APP_NAME,
    session_service=session_service,  # 与 Section 3 相同的服务
    memory_service=memory_service,
)

print("Runner 创建完成。")

Runner 创建完成。


In [39]:
# 测试 1：告诉 Agent 关于礼物的事（第一次对话）
# 回调会在轮次完成时自动将其保存到记忆
await run_session(
    auto_runner,
    "我在侄子一岁生日时送了他一个新的熊猫毛绒玩具！",
    "auto-save-test",
)

# 测试 2：在新会话中询问礼物（第二次对话）
# Agent 应该使用 preload_memory 检索记忆并正确回答
await run_session(
    auto_runner,
    "我送了侄子什么礼物？",
    "auto-save-test-2",  # 不同的会话 ID - 证明记忆跨会话工作！
)


### 会话: auto-save-test

用户 > 我在侄子一岁生日时送了他一个新的熊猫毛绒玩具！
模型 > 太棒了！熊猫毛绒玩具是很可爱的礼物，相信你的侄子一定会很喜欢的！一岁生日是宝宝成长中的一个重要里程碑，送上这样一份充满爱意的礼物，很有意义。

你选择的这个熊猫毛绒玩具有什么特别之处吗？比如是特别柔软的，还是有什么互动功能？

### 会话: auto-save-test-2

用户 > 我送了侄子什么礼物？
模型 > 为了回答这个问题，我需要您提供更多信息。您能告诉我您侄子的年龄、兴趣爱好，以及您通常会送他什么样的礼物吗？

如果您之前告诉我过，请提醒我一下，这样我才能回忆起来。


**刚刚发生了什么：**

1. **第一次对话：** 提到给侄子的礼物
   - 回调自动保存到记忆 ✓
2. **第二次对话（新会话）：** 询问礼物
   - `preload_memory` 自动检索记忆 ✓
   - Agent 正确回答 ✓

**零手动记忆调用！** 这就是自动化记忆管理的实际效果。

### 6.5 应该多久保存一次会话到记忆？

**选项：**

| 时机 | 实现方式 | 适用场景 |
|------|----------|----------|
| **每轮后** | `after_agent_callback` | 实时记忆更新 |
| **对话结束时** | 会话结束时手动调用 | 批量处理，减少 API 调用 |
| **定期间隔** | 基于定时器的后台任务 | 长时间运行的对话 |

---
## Section 7: 记忆整合（Memory Consolidation）

### 7.1 原始存储的局限性

**到目前为止我们存储的内容：**
- 每条用户消息
- 每条 Agent 响应
- 每次工具调用

**问题：**
```
会话：50 条消息 = 10,000 tokens
记忆：存储所有 50 条消息
搜索：返回所有 50 条消息 --> Agent 必须处理 10,000 tokens
```

这无法扩展。我们需要**整合**。

### 7.2 什么是记忆整合？

**记忆整合** = 仅提取**重要事实**，丢弃对话噪音。

**整合前（原始存储）：**

```
用户："我最喜欢的颜色是蓝绿色。我也喜欢紫色。
       其实，我大多数时候更喜欢蓝绿色。"
Agent："好的！我会记住的。"
用户："谢谢！"
Agent："不客气！"

--> 存储所有 4 条消息（冗余、冗长）
```

**整合后：**

```
提取的记忆："用户最喜欢的颜色：蓝绿色"

--> 存储 1 条简洁的事实
```

**好处：** 更少的存储空间、更快的检索、更准确的答案。

![记忆整合](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/memory-consolidation.png)

### 7.3 整合如何工作（概念）

**处理流程：**

```
1. 原始会话事件
   |
   v
2. LLM 分析对话
   |
   v
3. 提取关键事实
   |
   v
4. 存储简洁的记忆
   |
   v
5. 与现有记忆合并（去重）
```

**示例转换：**

```
输入："我对花生过敏。我不能吃任何含坚果的东西。"

输出：Memory {
  过敏："花生、树坚果"
  严重程度："完全避免"
}
```

自然语言 --> 结构化、可操作的数据。

### 7.4 记忆整合的下一步

**关键点：** 托管记忆服务**自动**处理整合。

**你使用相同的 API：**
- `add_session_to_memory()` <-- 相同的方法
- `search_memory()` <-- 相同的方法

**区别在于：** 幕后发生的事情。
- **InMemoryMemoryService：** 存储原始事件
- **VertexAiMemoryBankService：** 存储前进行智能整合

**更多学习：**
- [Vertex AI Memory Bank: 记忆整合指南](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/generate-memories) -> 你将在 Day 5 探索这个！

---
## 总结

你已经学会了 ADK 中 Memory 的**核心机制**：

1. **添加 Memory**
   - 与 `SessionService` 一起初始化 `MemoryService`
   - 两个服务都提供给 `Runner`

2. **存储信息**
   - `await memory_service.add_session_to_memory(session)`
   - 将会话数据转移到长期存储
   - 可以使用回调自动化

3. **搜索记忆**
   - `await memory_service.search_memory(app_name, user_id, query)`
   - 返回过去对话中的相关记忆

4. **在 Agent 中检索**
   - **被动式：** `load_memory` 工具（Agent 决定何时使用记忆）
   - **主动式：** `preload_memory` 工具（始终将记忆加载到 LLM 的系统指令中）

5. **记忆整合**
   - 从会话数据中提取关键信息
   - 由 Vertex AI Memory Bank 等托管记忆服务提供

## 恭喜！你已经学会了 ADK 中的 Memory 记忆管理！

**更多学习资源：**
- [ADK Memory 文档](https://google.github.io/adk-docs/sessions/memory/)
- [Vertex AI Memory Bank](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/overview)
- [记忆整合指南](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/generate-memories)

**下一步：**

准备好学习 Day 4 了吗？学习如何**实现可观测性并评估你的 Agent**，以确保它们在生产环境中按预期工作！